##### Data: ria_1
##### Experiment: ex_1

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm_notebook
import os
import numpy as np
from datetime import datetime
pd.set_option('display.max_colwidth', -1)

In [4]:
# #to Google Collab
# from google.colab import drive
# drive.mount('/content/gdrive')
# path_to_storage = '/content/gdrive/My Drive/UCU-diploma/openNMT'

In [5]:
# !git clone https://github.com/OpenNMT/OpenNMT-py.git OpenNMT
# !cd ./OpenNMT    
# !pip install -r requirements.txt

In [8]:
dir_to_all = './data/opennmt/ria_1_sources'
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
create_dir(dir_to_all)

In [60]:
"data/ria_news_dataset/ria.json.gz"
#!ls -la data/ria_news_dataset/
!head "./data/ria_news_dataset/ria.json.gz"

version https://git-lfs.github.com/spec/v1
oid sha256:59a25bbcb14a261a82ac6baa84d11d23ca2125c28f0cc1905418b5301a8cb12d
size 1025015063


In [6]:
def clean_news_text(text):
    return  re.sub("(&#(?:\d)*;)", "", re.sub("<.*?>", "", text)) \
    .replace("\n","").replace("\r","").replace("\x96","") \
    .replace("&nbsp;", "").replace("&mdash;", "")

In [65]:
#news = pd.read_json("./data/ria_news_dataset/ria.json.gz", lines=True,  dtype=['object', 'object'], compression='gzip')
news.text = news.text.apply(clean_news_text)

In [66]:
news.head(1)

,text,title
0,"москва, 31 янв - риа новости. большая часть из 33 детей, которых граждане сша пытались вывезти из гаити в организованный в доминиканской республике приют, не являются сиротами, сообщает в воскресенье агентство франс пресс со ссылкой на заявление представителя международной организации ""детские деревни sos"" (sos children's village), оказывающей помощь детям, оставшимся без родителейкак заявила агентству патрисия варгас (patricia vargas), курирующая программы ""детских деревень sos"" в центральной америке, мексике и на карибах, поговорив с детьми она выяснила, что родители многих из них живы. некоторые дети смогли назвать свои домашние адреса и номера телефонов, что дает возможность связаться с их родителями.в это воскресенье гаитянская полиция задержала десятерых граждан сша, подозреваемых в попытке без разрешения вывезти более 30 детей в доминиканскую республику.представитель баптистской церкви в городе меридиан американского штата айдахо шон лэнкфорд (sean lankford) заявил, что задержанные прибыли на гаити в составе группы, помогающей детям, которые остались без родителей после разрушительного землетрясения 12 января.лэнкфорд также сообщил, что в числе задержанных его дочь и жена, и они думали, что у них имеются все необходимые документы, позволяющие вывезти детей в организованный в доминиканской республике приют.в настоящее время все эти дети, за исключением маленькой девочки, страдающей от истощения, которая была госпитализирована, находятся в благотворительном центре организации в городе круа-де-букет (croix des bouquets), расположенном в 12 километрах к северо-востоку от столицы гаити порт-о-пренса.по словам варгас, точный возраст малышки не известен, врачи полагают, что ей около 7 месяцев.центр ""детских деревень sos"" на гаити юридически не является сиротским приютом и не отдает детей на усыновление.ранее гаитянские интернет-ресурсы сообщали, что за детьми, оставшимися сиротами после землетрясения, охотятся педофилы и торговцы людьми &gt;&gt;как отмечает франс пресс, после разгула стихии на гаити были установлены новые правила усыновления, согласно которым премьер-министр страны жан-макс бельрив должен лично разрешить вывоз сирот. целью подобных мер является пресечение попыток незаконного вывоза детей для преступных целей в условиях хаоса, царящего в стране после разгула стихии.по данным ответственных лиц на гаити, тысячи детей могли быть разлучены с родителями или лишиться их в результате двух землетрясений магнитудой 7 и 5,9, произошедших у побережья этого островного государства 12 января.","большинство детей, которых пытались увезти в сша из гаити, не сироты"


In [67]:
news.shape

(1003869, 2)

In [68]:
X_train, X_test = train_test_split(news, test_size=0.1, random_state=42)
X_train, X_val = train_test_split(X_train, test_size=0.11, random_state=42)
print("Len X_train: ", len(X_train))
print("Len X_val: ", len(X_val))
print("Len X_test: ", len(X_test))

Len X_train:  804098
Len X_val:  99384
Len X_test:  100387


In [69]:
def create_dataset(X, type_ds, to_dir):
    assert type_ds in ['test','train', 'valid']
    create_dir(to_dir)
    count_lines = 0
    src_file = "%s/%s_src.txt"%(to_dir,type_ds)
    tgt_file = "%s/%s_tgt.txt"%(to_dir,type_ds)
    with(open(src_file, 'w')) as f_text:
        with(open(tgt_file, 'w')) as f_titles:
            for i, row in tqdm_notebook(X.iterrows()):
                f_text.write(row.text + "\n")
                f_titles.write(row.title + "\n")
                count_lines +=1
    print("Source file: ", src_file)
    print("Target file: ", tgt_file)
    print("Count lines: ",count_lines)

In [70]:
print("Build train dataset")
create_dataset(X_train, 'train', "./data/opennmt/ria_1_sources")
print("Build test dataset")
create_dataset(X_test, 'test', "./data/opennmt/ria_1_sources")
print("Build valid dataset")
create_dataset(X_val, 'valid', "./data/opennmt/ria_1_sources")

Build train dataset



Source file:  ./data/opennmt/ria_1_sources/train_src.txt
Target file:  ./data/opennmt/ria_1_sources/train_tgt.txt
Count lines:  804098
Build test dataset



Source file:  ./data/opennmt/ria_1_sources/test_src.txt
Target file:  ./data/opennmt/ria_1_sources/test_tgt.txt
Count lines:  100387
Build valid dataset



Source file:  ./data/opennmt/ria_1_sources/valid_src.txt
Target file:  ./data/opennmt/ria_1_sources/valid_tgt.txt
Count lines:  99384


In [71]:
!wc -l ./data/opennmt/ria_1_sources/train_src.txt
!wc -l ./data/opennmt/ria_1_sources/train_tgt.txt
!wc -l ./data/opennmt/ria_1_sources/test_src.txt
!wc -l ./data/opennmt/ria_1_sources/test_tgt.txt
!wc -l ./data/opennmt/ria_1_sources/valid_src.txt
!wc -l ./data/opennmt/ria_1_sources/valid_tgt.txt

  804098 ./data/opennmt/ria_1_sources/train_src.txt
  804098 ./data/opennmt/ria_1_sources/train_tgt.txt
  100387 ./data/opennmt/ria_1_sources/test_src.txt
  100387 ./data/opennmt/ria_1_sources/test_tgt.txt
   99384 ./data/opennmt/ria_1_sources/valid_src.txt
   99384 ./data/opennmt/ria_1_sources/valid_tgt.txt


In [ ]:
#create_dir("./data/opennmt/ria_1_sources")
create_dir("./data/opennmt/ria_1_sources/data")

In [58]:
! python ./OpenNMT-py/preprocess.py \
-train_src ./data/opennmt/ria_1_sources/train_src.txt \
-train_tgt ./data/opennmt/ria_1_sources/train_tgt.txt \
-valid_src ./data/opennmt/ria_1_sources/valid_src.txt \
-valid_tgt ./data/opennmt/ria_1_sources/valid_tgt.txt \
-save_data ./data/opennmt/ria_1_sources/data/data \
-share_vocab \
-dynamic_dict \
-src_vocab_size 50000 \
-report_every 1000 \
-shard_size 80000


In [ ]:
! python ./OpenNMT/train.py \
-data "/content/gdrive/My Drive/UCU-diploma/openNMT/opennmt/ria_1_sources/data/data" \
-save_model "/content/gdrive/My Drive/UCU-diploma/openNMT/opennmt/ria_1_target/model" \
-share_embeddings \
-batch_type tokens \
-batch_size 64 \
-valid_batch_size 2 \
-copy_attn \
-global_attention mlp \
-word_vec_size 128 \
-rnn_size 512 \
-layers 1 \
-encoder_type brnn \
-max_grad_norm 2 \
-dropout 0. \
-optim adagrad \
-adagrad_accumulator_init 0.1 \
-reuse_copy_attn \
-copy_loss_by_seqlength \
-bridge \
-seed 42 \
-gpu_ranks 0 \
-save_checkpoint_steps 10000 \
-train_steps 100000 \
-learning_rate 0.001 \
-report_every 1000 \
-valid_steps 1000

In [ ]:
! python ./OpenNMT/translate.py \
-model "/content/gdrive/My Drive/UCU-diploma/openNMT/opennmt/data_shared/model_step_66000.pt" \
-src "/content/gdrive/My Drive/UCU-diploma/openNMT/opennmt/all/test_src.broad.txt" \
-tgt "/content/gdrive/My Drive/UCU-diploma/openNMT/opennmt/all/test_tgt (1).news.txt" \
-output "/content/gdrive/My Drive/UCU-diploma/openNMT/opennmt/all/pred.news.txt" \
-beam_size 10 \
-dynamic_dict \
-share_vocab \
-batch_size 2 \
-batch_type sents \
-gpu 0 \
-seed 42 \
--report_rouge



In [ ]:
files2rouge ./data/opennmt/all/pred_news.broad.txt ./data/opennmt/all/test_tgt.news.txt